In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import seaborn as sns
import SimpleITK as sitk
import matplotlib.backends.backend_pdf
import pandas as pd

#plt.rcParams['figure.figsize'] = [40, 30]

In [15]:
def volumetric_dice(img, mask):
    img[img > 1] = 1
    img[img < 1] = 0
    mask[mask > 1] = 1
    mask[mask < 1] = 0
    return np.sum(mask[img == mask]) * 2.0 / (np.sum(img)+np.sum(mask))

In [16]:
root_path = "/Users/amithkamath/repo/deep-planner"
data_path = os.path.join(root_path, "data", "dldp-perturb")
gt_path = os.path.join(data_path, "ground_truth")
perturbed_path = os.path.join(data_path, "Prediction")
viz_path = os.path.join(data_path, "visualization")
results_path = os.path.join(root_path, "results")

In [17]:
# select all cases in the folder
cases = sorted(glob.glob(os.path.join(perturbed_path, "*")))
case_names = []

df = pd.DataFrame({'Case': pd.Series(dtype='str'),
                   'Structure': pd.Series(dtype='str'),
                   'NumPts': pd.Series(dtype='int'),
                   'Size' : pd.Series(dtype='int'),
                   'AvgDiff': pd.Series(dtype='float'), 
                   'StdDiff' : pd.Series(dtype='float'), 
                   'XCorrGrad': pd.Series(dtype='float'), 
                   'XCorrDist': pd.Series(dtype='float')})

for case in cases:
    # select the case number
    case_nr = case.split("/")[-1]
    case_names.append(case_nr)
    
    # select and read the ground truth dose
    dose_gt_path = glob.glob(os.path.join(perturbed_path, case_nr) + "/Dose_gt.nii.gz")
    dose_gt = sitk.ReadImage(dose_gt_path[0])
    dose_gt = sitk.GetArrayFromImage(dose_gt)
    dose_gradient = np.gradient(dose_gt)
    dose_grad_mag = np.sqrt(dose_gradient[0]**2 + dose_gradient[1]**2 + dose_gradient[2]**2)

    # select and read the predicted dose
    target_path = glob.glob(os.path.join(gt_path, case_nr) + "/Target.nii.gz")
    target_mask = sitk.ReadImage(target_path[0])
    target_mask = sitk.GetArrayFromImage(target_mask)
    target_locs = np.nonzero(target_mask)

    # select all the structures in the folder
    structures = glob.glob(os.path.join(perturbed_path, case_nr) + "/Perturbed_*")

    # For every structure calculate the DVHs and dose metrics
    for structure in structures:
        # select the name of the structure
        str_name = structure.split("/")[-1].split(".")[0].replace("Perturbed_", "")
        struct = sitk.ReadImage(structure)
        perturbation_vol = sitk.GetArrayFromImage(struct)
        locs = np.nonzero(perturbation_vol)

        gt_struct_path = os.path.join(gt_path, case_nr, str_name + ".nii.gz")
        gt_struct = sitk.ReadImage(gt_struct_path)
        gt_struct_vol = sitk.GetArrayFromImage(gt_struct)
        gt_size = len(np.nonzero(gt_struct_vol)[0])

        diff = []
        grad = []
        min_dist = []

        for idx in range(len(locs[0])):
            absdiff_dose = perturbation_vol[locs[0][idx]][locs[1][idx]][locs[2][idx]]
            grad_dose = dose_grad_mag[locs[0][idx]][locs[1][idx]][locs[2][idx]]
            
            min_target_dist = np.sqrt((locs[0][idx] - target_locs[0][0])**2 + (locs[1][idx] - target_locs[1][0])**2 + (locs[2][idx] - target_locs[2][0])**2)
            for tidx in range(len(target_locs[0])):
                new_dist = np.sqrt((locs[0][idx] - target_locs[0][tidx])**2 + (locs[1][idx] - target_locs[1][tidx])**2 + (locs[2][idx] - target_locs[2][tidx])**2)
                if new_dist < min_target_dist:
                    min_target_dist = new_dist
            
            diff.append(absdiff_dose)
            grad.append(grad_dose)
            min_dist.append(min_target_dist)
        
        df_data = {"diff": diff, "grad": grad, "dist": min_dist}

        df = df.append({'Case' : case_nr, 
                        'Structure' : str_name, 
                        'NumPts' : len(locs[0]),
                        'Size' : gt_size,  
                        'AvgDiff' : np.mean(diff), 
                        'StdDiff' : np.std(diff), 
                        'XCorrGrad' : np.corrcoef(diff, grad)[0][1],
                        'XCorrDist' : np.corrcoef(diff, min_dist)[0][1]}, 
                        ignore_index = True)

        subject_data = pd.DataFrame.from_dict(df_data)
        subject_data.to_csv(os.path.join(root_path, "results", str(case_nr) + "_" + str(str_name) + ".csv"))

df.to_csv(os.path.join(results_path, "perturbation_data_new.csv"))

/var/folders/yr/q_0gpqgs1lq_rtbttpk3n9bh0000gn/T/ipykernel_58383/303220411.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Case' : case_nr,
/var/folders/yr/q_0gpqgs1lq_rtbttpk3n9bh0000gn/T/ipykernel_58383/303220411.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Case' : case_nr,
/var/folders/yr/q_0gpqgs1lq_rtbttpk3n9bh0000gn/T/ipykernel_58383/303220411.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Case' : case_nr,
/var/folders/yr/q_0gpqgs1lq_rtbttpk3n9bh0000gn/T/ipykernel_58383/303220411.py:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Case' : case_